# SBTi Temperature Alignment tool - Reporting
This notebook ...

#### Setting up
Imports SBTi tools, import data providers, and load the portfolio.

In [ ]:
# Only required if you didn't install the module yet
!pip install SBTi

In [1]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
%aimport -pandas
import pandas as pd

In [2]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/example_portfolio.csv", "example_portfolio.csv")

In [3]:
def anonymize(portfolio, provider):
    portfolio_companies = portfolio['company_name'].unique()
    for index, company_name in enumerate(portfolio_companies):
        portfolio.loc[portfolio['company_name'] == company_name, 'company_id'] = 'C' + str(index + 1)
        portfolio.loc[portfolio['company_name'] == company_name, 'company_isin'] = 'C' + str(index + 1)
        provider.data['fundamental_data'].loc[provider.data['fundamental_data']['company_name'] == company_name, 'company_id'] = 'C' + str(index + 1)
        provider.data['fundamental_data'].loc[provider.data['fundamental_data']['company_name'] == company_name, 'company_isic'] = 'C' + str(index + 1)
        provider.data['target_data'].loc[provider.data['target_data']['company_name'] == company_name, 'company_id'] = 'C' + str(index + 1)
        portfolio.loc[portfolio['company_name'] == company_name, 'company_name'] = 'Company' + str(
            index + 1)
        provider.data['fundamental_data'].loc[provider.data['fundamental_data']['company_name'] == company_name, 'company_name'] = 'Company' + str(
                    index + 1)
        provider.data['target_data'].loc[provider.data['target_data']['company_name'] == company_name, 'company_name'] = 'Company' + str(
                    index + 1)
    for index, company_name in enumerate(provider.data['fundamental_data']['company_name'].unique()):
        if company_name not in portfolio['company_name'].unique():
            provider.data['fundamental_data'].loc[provider.data['fundamental_data']['company_name'] == company_name, 'company_id'] = '_' + str(index + 1)
            provider.data['fundamental_data'].loc[provider.data['fundamental_data']['company_name'] == company_name, 'company_name'] = 'Company_' + str(
                                index + 1)
    return portfolio, provider

In [6]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")
portfolio = pd.read_csv("example_portfolio.csv", encoding="iso-8859-1")
portfolio["company_isin"] = portfolio.ISIN
portfolio, provider = anonymize(portfolio, provider)
companies = [SBTi.interfaces.PortfolioCompany.parse_obj(company) for company in portfolio.to_dict(orient="records")]
portfolio_data = SBTi.utils.get_data([provider], companies)

### Compute temperature scores
Configure computation settings

In [7]:
time_frames = list(SBTi.interfaces.ETimeFrames)
scopes = [EScope.S1S2, EScope.S3, EScope.S1S2S3]
fallback_score = 3.2
model = 4
aggregation_method = PortfolioAggregationMethod.WATS
grouping = None

In [8]:
temperature_score = TemperatureScore(
    time_frames=time_frames,
    scopes=scopes,
    fallback_score=fallback_score,
    model=model,
    aggregation_method=aggregation_method,
    grouping=grouping
)

In [9]:
amended_portfolio = temperature_score.calculate(portfolio_data)
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)

## Save anonymized portfolio data

In [5]:
portfolio_data.to_clipboard(excel=True, index=False)

Save anonymized data provider data - seems to be included in portfolio_data dataframe

Save applied settings for computations